In [2]:
from brainlit.utils.session import NeuroglancerSession
from brainlit.utils.swc import graph_to_paths
import napari

/Users/johnduva/opt/anaconda3/envs/neuroData3_7/lib/python3.7/site-packages/python_jsonschema_objects/__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


In [96]:
dir = "s3://open-neurodata/brainlit/brain1"
dir_segments = "s3://open-neurodata/brainlit/brain1_segments"

# Create a list of bboxes for each cell in 'dir' using 'pull_voxel'
cells = [];
ngl_sess = NeuroglancerSession(url=dir, url_segments=dir_segments, mip=1)
for i in range(1000):
    try:
        # pull_voxel(seg_id, v_id, radius)
        img, bbox, idx = ngl_sess.pull_voxel(i, 0, 15) 
        cells.append(bbox)
    except:
        pass

Downloading: 100%|██████████| 1/1 [00:00<00:00, 29.56it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 15.23it/s]


Downloading:   0%|          | 0/4 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 30.44it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  6.99it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 30.80it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 31.70it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00, 30.23it/s]


Downloading:   0%|          | 0/2 [00:00<?, ?it/s]


In [134]:
import csv
with open('cellCenters.csv', mode='w') as employee_file:
    for i in range(0, len(cells)):
        cntr = cells[i].center()
        employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        employee_writer.writerow([i, cntr[0], cntr[1], cntr[2]])

In [135]:
import pandas as pd
df = pd.read_csv('cellCenters.csv', header=None)
df.rename(columns={0: 'Id',  1: 'Center of Mass (Geometry): X (px)', 2: 'Center of Mass (Geometry): Y (px)', 3: 'Center of Mass (Geometry): Z (px)'}, inplace=True)
df.to_csv('cellCenters.csv', index=False) # save to new csv file